<a href="https://colab.research.google.com/github/akajammythakkar/call_center_analysis/blob/main/Call_Center_Analysis_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [10]:
# To run this code you need to install:
! pip install google-genai

In [11]:
!rm -rf cr

In [12]:
!unzip '/content/Call Recordings-20250825T130307Z-1-001.zip' -d 'cr'

Archive:  /content/Call Recordings-20250825T130307Z-1-001.zip
  inflating: cr/Call Recordings/Medical_FP_Outbound_I.wav  
  inflating: cr/Call Recordings/Insurance I.wav  
  inflating: cr/Call Recordings/Sales_Retail I.wav  
  inflating: cr/Call Recordings/Medical_FP_Inbound_II.wav  
  inflating: cr/Call Recordings/Sales_Real Estate  I.wav  
  inflating: cr/Call Recordings/Inbound_Medical_I.wav  
  inflating: cr/Call Recordings/Medical_FP_Outbound_II.wav  
  inflating: cr/Call Recordings/Sales_Real Estate  II.wav  
  inflating: cr/Call Recordings/E-commerce II.wav  
  inflating: cr/Call Recordings/Hindi Banking_CC_I.WAV  
  inflating: cr/Call Recordings/Medical_II.wav  
  inflating: cr/Call Recordings/Banking I.wav  
  inflating: cr/Call Recordings/Banking II.wav  
  inflating: cr/Call Recordings/Hindi Banking_CC_II.WAV  
  inflating: cr/Call Recordings/Medical_FP_Inbound_I.wav  
  inflating: cr/Call Recordings/Sales_Retail II.wav  
  inflating: cr/Call Recordings/E-commerce I.wav  
  

In [13]:
# To run this code you need to install:
# pip install google-genai

import os
from google import genai
from google.genai import types
from google.colab import userdata

def generate(file_path):
    client = genai.Client(
        api_key=userdata.get("GEMINI_KEY"),
    )

    model = userdata.get("MODEL_NAME")

    # Read audio file directly from your path
    with open(file_path, "rb") as f:
        audio_bytes = f.read()

    audio_part = types.Part.from_bytes(
        mime_type="audio/wav",
        data=audio_bytes,
    )

    prompt_text = """
You are an AI assistant analyzing **call center conversation transcripts**.
Your task is to carefully extract structured details, summarize the interaction, and rate the call center executive.
Always output results in a structured JSON format so they can be exported to Excel.

If you are unable to confidently determine any field, output `"I don't understand"` instead of making assumptions.

----------

### **Instructions**

1.  **Classification & Call Metadata**

    -   Identify the **industry domain** of the call (e.g., Banking, Telecom, Insurance, Retail, Healthcare, etc.).

    -   Classify the **call type**:

        -   _Inbound_: Customer called the call center.

        -   _Outbound_: Call center initiated the call.

    -   Extract **all participants' names** mentioned in the transcript.

    -   Detect the **gender** of each participant (Male/Female/Unknown).

    -   Detect the **primary language** spoken in the call.

2.  **Call Timing Details**

    -   Total duration of the call (minutes:seconds).

    -   Speaking duration:

        -   How long the **customer** spoke.

        -   How long the **call center executive** spoke.

    -   Wait/Hold duration:

        -   Detect any period where the customer was on hold with background music.

3.  **Call Summary**

    -   **Query**: What was the customer’s issue or request?

    -   **Resolution Stated**: How the executive responded and what resolution was promised.

    -   **Resolution Time Suggested**: Timeframe committed by the executive (e.g., “24 hours”, “3 business days”).

4.  **Call Center Executive Evaluation**
    Rate the executive in **three categories** using the rubric below:

    -   **Politeness (1–5)**
        1 = Rude or dismissive
        2 = Neutral, minimal courtesy
        3 = Polite but not empathetic
        4 = Polite and moderately empathetic
        5 = Very polite, empathetic, customer-focused

    -   **Language Clarity (1–5)**
        1 = Very unclear, hard to understand
        2 = Broken or confusing language
        3 = Adequate, understandable but with flaws
        4 = Clear and fluent, minor issues
        5 = Excellent clarity, professional communication

    -   **Adherence to Process (1–5)**
        1 = Did not follow process at all
        2 = Major gaps in process adherence
        3 = Partially followed process, some gaps
        4 = Mostly followed process correctly
        5 = Fully adhered to standard process/policy


----------

### **Output Format (JSON)**

```json
{
  "Industry": "",
  "Call_Type": "",
  "Participants": [
    {"Name": "", "Role": "Customer/Executive", "Gender": "", "Language": ""}
  ],
  "Call_Timing": {
    "Total_Duration": "",
    "Customer_Speaking_Duration": "",
    "Executive_Speaking_Duration": "",
    "Wait_Time": ""
  },
  "Summary": {
    "Query": "",
    "Resolution_Stated": "",
    "Resolution_Time_Suggested": ""
  },
  "Executive_Rating": {
    "Politeness": 0,
    "Language_Clarity": 0,
    "Adherence": 0
  }
}

```
"""

    contents = [
        types.Content(
            role="user",
            parts=[audio_part, types.Part.from_text(text=prompt_text)],
        ),
    ]

    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]

    # Config with thinking ability + JSON output
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        response_mime_type="application/json"
        # tools = tools
    )

    result = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,



    )
    # print(result)
    return result



In [14]:
import os

files = []
for dirpath, dirnames, filenames in os.walk('/content/cr'):
    for filename in filenames:
        files.append(os.path.join(dirpath, filename))


In [15]:
files

['/content/cr/Call Recordings/Sales_Retail I.wav',
 '/content/cr/Call Recordings/Sales_Real Estate  I.wav',
 '/content/cr/Call Recordings/Medical_FP_Inbound_I.wav',
 '/content/cr/Call Recordings/Medical_II.wav',
 '/content/cr/Call Recordings/Sales_Real Estate  II.wav',
 '/content/cr/Call Recordings/Inbound_Medical_I.wav',
 '/content/cr/Call Recordings/E-commerce II.wav',
 '/content/cr/Call Recordings/Medical_FP_Outbound_I.wav',
 '/content/cr/Call Recordings/Sales_Retail II.wav',
 '/content/cr/Call Recordings/Medical_I.wav',
 '/content/cr/Call Recordings/Hindi Banking_CC_II.WAV',
 '/content/cr/Call Recordings/Medical_FP_Inbound_II.wav',
 '/content/cr/Call Recordings/Insurance I.wav',
 '/content/cr/Call Recordings/Banking II.wav',
 '/content/cr/Call Recordings/E-commerce I.wav',
 '/content/cr/Call Recordings/Inbound_Medical_II.wav',
 '/content/cr/Call Recordings/Banking I.wav',
 '/content/cr/Call Recordings/Medical_FP_Outbound_II.wav',
 '/content/cr/Call Recordings/Hindi Banking_CC_I.WAV

In [16]:
# result = generate('/content/cr/Call Recordings/Sales_Real Estate  I.wav')

# # after each model call:
# raw = result.text
# data = json.loads(raw)
# results.append(data)

In [17]:
# import json
# results = []

# for path in files:
#   try:
#     result = generate(path)
#     # print(result)
#     with open("file.txt", "a+") as f:
#       f.write(result.text)
#     # after each model call:
#     raw = result.text
#     data = json.loads(raw)
#     results.append(data)
#     print(f"completed for path: {path}")
#   except Exception as e:
#     print(f"Error in {path}: {e}")

In [18]:
import json
import time
results = []

for path in files:
    try:
        result = generate(path)

        # Parse JSON and add file path
        raw = result.text
        data = json.loads(raw)
        data["file_path"] = path

        # Write enriched JSON to file
        with open("file.txt", "a+") as f:
            f.write(json.dumps(data) + "\n")

        # Save result in memory
        results.append(data)
        print(f"completed for path: {path}")
        time.sleep(10)

    except Exception as e:
        time.sleep(10)
        result = generate(path)

        # Parse JSON and add file path
        raw = result.text
        data = json.loads(raw)
        data["file_path"] = path

        # Write enriched JSON to file
        with open("file.txt", "a+") as f:
            f.write(json.dumps(data) + "\n")

        # Save result in memory
        results.append(data)
        print(f"completed for path: {path}")
        print(f"Error in {path}: {e}")

completed for path: /content/cr/Call Recordings/Sales_Retail I.wav
completed for path: /content/cr/Call Recordings/Sales_Real Estate  I.wav
completed for path: /content/cr/Call Recordings/Medical_FP_Inbound_I.wav
completed for path: /content/cr/Call Recordings/Medical_II.wav
completed for path: /content/cr/Call Recordings/Sales_Real Estate  II.wav
completed for path: /content/cr/Call Recordings/Inbound_Medical_I.wav
completed for path: /content/cr/Call Recordings/E-commerce II.wav
completed for path: /content/cr/Call Recordings/Medical_FP_Outbound_I.wav
completed for path: /content/cr/Call Recordings/Sales_Retail II.wav
completed for path: /content/cr/Call Recordings/Medical_I.wav
completed for path: /content/cr/Call Recordings/Hindi Banking_CC_II.WAV
completed for path: /content/cr/Call Recordings/Medical_FP_Inbound_II.wav
completed for path: /content/cr/Call Recordings/Insurance I.wav
completed for path: /content/cr/Call Recordings/Banking II.wav
completed for path: /content/cr/Call 

In [19]:
# import json
# # results = []

# for path in ['/content/cr/Call Recordings/Medical_FP_Outbound_I.wav']:#files:
#     try:
#         result = generate(path)

#         # Parse JSON and add file path
#         raw = result.text
#         data = json.loads(raw)
#         data["file_path"] = path

#         # Write enriched JSON to file
#         with open("file.txt", "a+") as f:
#             f.write(json.dumps(data) + "\n")

#         # Save result in memory
#         results.append(data)
#         print(f"completed for path: {path}")
#     except Exception as e:
#         print(f"Error in {path}: {e}")


In [20]:
import pandas as pd

def flatten_record(r: dict) -> dict:
    row = {
        "File_Path": r.get("file_path", ""),
        "Industry": r.get("Industry", ""),
        "Call_Type": r.get("Call_Type", ""),
        "Total_Duration": r.get("Call_Timing", {}).get("Total_Duration", ""),
        "Customer_Speaking_Duration": r.get("Call_Timing", {}).get("Customer_Speaking_Duration", ""),
        "Executive_Speaking_Duration": r.get("Call_Timing", {}).get("Executive_Speaking_Duration", ""),
        "Wait_Time": r.get("Call_Timing", {}).get("Wait_Time", ""),
        "Query": r.get("Summary", {}).get("Query", ""),
        "Resolution_Stated": r.get("Summary", {}).get("Resolution_Stated", ""),
        "Resolution_Time_Suggested": r.get("Summary", {}).get("Resolution_Time_Suggested", ""),
        "Politeness": r.get("Executive_Rating", {}).get("Politeness", ""),
        "Language_Clarity": r.get("Executive_Rating", {}).get("Language_Clarity", ""),
        "Adherence": r.get("Executive_Rating", {}).get("Adherence", "")
    }

    # expand participants as separate columns
    for i, p in enumerate(r.get("Participants", []), start=1):
        row[f"Participant_{i}_Name"] = p.get("Name", "")
        row[f"Participant_{i}_Role"] = p.get("Role", "")
        row[f"Participant_{i}_Gender"] = p.get("Gender", "")
        row[f"Participant_{i}_Language"] = p.get("Language", "")
    return row

# flatten all results
rows = [flatten_record(r) for r in results]

# convert to DataFrame
df = pd.DataFrame(rows)

# export
df.to_csv("call_analysis_1.csv", index=False)
df.to_excel("call_analysis_1.xlsx", index=False)

print("Exported call_analysis.csv and call_analysis.xlsx ✅")


Exported call_analysis.csv and call_analysis.xlsx ✅


In [28]:
df['File_Path'] = df['File_Path'].apply(lambda x: x.replace('/content/cr/Call Recordings/', ''))

In [29]:
display(df)

,File_Path,Industry,Call_Type,Total_Duration,Customer_Speaking_Duration,Executive_Speaking_Duration,Wait_Time,Query,Resolution_Stated,Resolution_Time_Suggested,...,Participant_1_Gender,Participant_1_Language,Participant_2_Name,Participant_2_Role,Participant_2_Gender,Participant_2_Language,Participant_3_Name,Participant_3_Role,Participant_3_Gender,Participant_3_Language
0,Sales_Retail I.wav,IT Services,Outbound,05:59,02:21,03:07,00:00,The customer inquired about setting up an IVR ...,The executive confirmed that the requested IVR...,Same Day (A product demonstration was schedule...,...,Male,Hindi,Executive,Executive,Male,Hindi,NaN,NaN,NaN,NaN
1,Sales_Real Estate I.wav,Real Estate,Outbound,03:51,00:58,02:45,00:00,The call was an outbound sales call. The custo...,The executive provided the price of the 2 BHK ...,Immediately,...,Female,Hindi,I don't understand,Customer,Male,Hindi,NaN,NaN,NaN,NaN
2,Medical_FP_Inbound_I.wav,Healthcare,Inbound,05:20,00:48,03:52,00:00,The customer was referred by a doctor and call...,The executive explained various contraceptive ...,Not Applicable,...,Male,Hindi,Shahrukh,Customer,Male,Hindi,NaN,NaN,NaN,NaN
3,Medical_II.wav,Healthcare,Inbound,04:07,01:21,02:15,00:00,The customer called to book a health test for ...,The executive successfully booked the health t...,Sample collection is scheduled for the next mo...,...,Male,Hindi,Not Mentioned,Executive,Female,Hindi,Anita Gupta,Customer,Female,Hindi
4,Sales_Real Estate II.wav,Real Estate,Outbound,3:33,0:53,2:40,0:00,The customer inquired about the details of a 2...,The executive provided all the details about t...,Immediately,...,Male,Hindi,Surendra,Customer,Male,Hindi,NaN,NaN,NaN,NaN
5,Inbound_Medical_I.wav,Healthcare,Inbound,04:38,00:41,03:45,00:00,The customer called to inquire about different...,The executive explained several family plannin...,Not Applicable,...,Female,Hindi,Sonia,Customer,Female,Hindi,NaN,NaN,NaN,NaN
6,E-commerce II.wav,Retail,Outbound,11:22,04:16,06:24,00:42,The executive called to confirm the delivery a...,The executive successfully verified the custom...,24 to 48 hours,...,Female,Hindi,Pankaj Kumar Thakur,Customer,Male,Hindi,NaN,NaN,NaN,NaN
7,Medical_FP_Outbound_I.wav,Healthcare,Outbound,05:16,01:02,03:57,00:17,The executive made an outbound call to provide...,The executive explained various family plannin...,N/A,...,Female,Hindi,Krishna,Customer,Male,Hindi,Krishna Devi,Customer,Female,Hindi
8,Sales_Retail II.wav,Telecom,Inbound,01:31,00:39,00:52,00:00,The customer called to inquire about WhatsApp ...,"The executive explained the IVR services, incl...",No specific time frame was committed for the W...,...,Male,Hindi,Yashas Agarwal,Customer,Male,Hindi,NaN,NaN,NaN,NaN
9,Medical_I.wav,Healthcare,Outbound,04:05,00:41,02:29,00:00,The customer received a message about a health...,The executive explained that the package inclu...,Sample collection the next morning; reports by...,...,Male,Hindi,Virender Yadav,Customer,Male,Hindi,Gita Yadav,Customer,Female,Hindi
